In [ ]:
!pip install transformers==2.11.0

     |████████████████████████████████| 675kB 17.1MB/s 
     |████████████████████████████████| 1.2MB 51.4MB/s 
     |████████████████████████████████| 890kB 60.6MB/s 
     |████████████████████████████████| 3.8MB 55.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=66578faddbd0ddd8bd15111bee5e29b5df864f7cc61d398502c26a5e73edbe5e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!pip install ipython-autotime

%load_ext autotime

time: 156 µs (started: 2021-02-13 12:53:25 +00:00)


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers

time: 5.48 s (started: 2021-02-13 12:53:25 +00:00)


In [ ]:
print(transformers.__version__)

2.11.0
time: 1.37 ms (started: 2021-02-13 12:53:31 +00:00)


In [ ]:
import pickle
import tempfile
from tensorflow.keras.models import Sequential, load_model, save_model, Model
from tensorflow.keras.layers import Dense

# Hotfix function
def make_keras_picklable():
    def __getstate__(self):
        model_str = ""
        with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
            save_model(self, fd.name, overwrite=True)
            model_str = fd.read()
        d = {'model_str': model_str}
        return d

    def __setstate__(self, state):
        with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
            fd.write(state['model_str'])
            fd.flush()
            model = load_model(fd.name)
        self.__dict__ = model.__dict__


    cls = Model
    cls.__getstate__ = __getstate__
    cls.__setstate__ = __setstate__

# Run the function
make_keras_picklable()

time: 10.8 ms (started: 2021-02-13 12:58:06 +00:00)


In [ ]:
train_df = pd.read_csv("train.csv", error_bad_lines=False)

print(f"Total train samples : {train_df.shape[0]}")

Total train samples : 404290
time: 739 ms (started: 2021-02-13 12:58:11 +00:00)


In [ ]:
train_df
# 1 in 'is_duplicate' denotes that the questions are similar

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
404285,404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,404287,537928,537929,What is one coin?,What's this coin?,0
404288,404288,537930,537931,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


time: 30 ms (started: 2021-02-13 12:58:12 +00:00)


In [ ]:
train_df.drop(['id', 'qid1', 'qid2'], axis=1, inplace=True)

train_df.columns = ['question1', 'question2', 'label']

time: 30.6 ms (started: 2021-02-13 12:58:12 +00:00)


In [ ]:
train_df

,question1,question2,label
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...
404285,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,What is one coin?,What's this coin?,0
404288,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


time: 36 ms (started: 2021-02-13 12:58:12 +00:00)


In [ ]:
train, validate, test = \
              np.split(train_df.sample(frac=1, random_state=42), 
                       [int(.6*len(train_df)), int(.8*len(train_df))])

time: 172 ms (started: 2021-02-13 12:58:14 +00:00)


In [ ]:
train

,question1,question2,label
8067,How do I play Pokémon GO in Korea?,How do I play Pokémon GO in China?,0
368101,What are some of the best side dishes for crab...,What are some good side dishes for buffalo chi...,0
70497,Which is more advisable and better material fo...,What is the best server setup for buddypress?,0
226567,How do I improve logical programming skills?,How can I improve my logical skills for progra...,1
73186,How close we are to see 3rd world war?,How close is a World War III?,1
...,...,...,...
114191,What Is a Blogger?,What is blogger?,1
178082,Will a small humidifier near the bed in a larg...,I have to design a large humidity controlled s...,0
312122,Is there any Nano technology GPS tracking feat...,The Newly Introduced 2000 Rupees and 500 Rupee...,1
133284,What are the best restaurants to try when visi...,What are the best restaurants to try when visi...,0


time: 35.2 ms (started: 2021-02-13 12:58:15 +00:00)


In [ ]:
validate

,question1,question2,label
106199,What are Sodexo coupons? Why does a company gi...,Why don't eCommerce sites accept Sodexo/Ticket...,0
83886,Can pet rats eat peanut butter?,Can puppies eat peanut butter? Why or why not?,0
200543,Can IAS women wear shirt and trousers to work?,Are IAS officiers required to wear uniform?,0
265070,Got any ideas for pink clothing?,Where should I shop for clothes?,0
211647,What can be the maximum share of a single pers...,What's the STCG and LTCG tax rate on shares fo...,0
...,...,...,...
191549,"As a venture capital investor, if you had an o...","As a venture capital investor, if you had an o...",0
281839,What is the best thing that ever happened to y...,What is the most interesting thing that ever h...,0
369326,How can I be an accountant?,How do I become an accountant?,0
119616,Which construction technology is used for Quat...,How does it feels to be on the top of Qutub Mi...,0


time: 23.6 ms (started: 2021-02-13 12:58:15 +00:00)


In [ ]:
test

,question1,question2,label
373745,How is the interaction when a person with psyc...,Do you prefer a textual conversation or a face...,0
178451,Time Travel Is It Possible?,Do you think time travel is possible?,1
83264,Can we use wireless/Bluetooth earphones in fli...,How can I use a wireless PS3 controller using ...,0
384938,"Once I become an accredited investor, am I alw...",How do you become an accredited investor?,0
282126,In which country we get copper scrap cheap?,I have written upsc prelims 2016. I'm getting ...,0
...,...,...,...
259178,How do you feed a Venus flytrap?,Do Venus flytraps have feelings?,0
365838,I'm an Electrical Engineering student of Sixth...,I am a electrical engineering final year stude...,1
131932,What is the best DSLR camera for a beginner to...,What's the best DSLR camera on a tight budget?,1
146867,What is a good computer desk for dual monitors?,What are the best desks for dual monitors?,0


time: 22.4 ms (started: 2021-02-13 12:58:15 +00:00)


In [ ]:
print("Train Target Distribution")
print(train.label.value_counts())

Train Target Distribution
0    152849
1     89725
Name: label, dtype: int64
time: 5.65 ms (started: 2021-02-13 12:58:15 +00:00)


In [ ]:
print("Validate Target Distribution")
print(validate.label.value_counts())

Validate Target Distribution
0    51236
1    29622
Name: label, dtype: int64
time: 5.84 ms (started: 2021-02-13 12:58:15 +00:00)


In [ ]:
print("Test Target Distribution")
print(test.label.value_counts())

Test Target Distribution
0    50942
1    29916
Name: label, dtype: int64
time: 7.3 ms (started: 2021-02-13 12:58:15 +00:00)


In [ ]:
y_train = tf.keras.utils.to_categorical(train.label, num_classes=2)
y_validate = tf.keras.utils.to_categorical(validate.label, num_classes=2)
y_test = tf.keras.utils.to_categorical(test.label, num_classes=2)

time: 11.9 ms (started: 2021-02-13 12:58:15 +00:00)


In [ ]:
max_length = 128  # Maximum length of input sentence to the model.
batch_size = 32
epochs = 2

time: 5.6 ms (started: 2021-02-13 12:58:16 +00:00)


In [ ]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

time: 26.8 ms (started: 2021-02-13 12:58:16 +00:00)


In [ ]:
# Create the model under a distribution strategy scope.
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Encoded token ids from BERT tokenizer.
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )
    # Loading pretrained BERT model.
    bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = False

    sequence_output, pooled_output = bert_model.bert(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
    bi_lstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, return_sequences=True)
    )(sequence_output)
    # Applying hybrid pooling approach to bi_lstm sequence output.
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    concat = tf.keras.layers.concatenate([avg_pool, max_pool])
    dropout = tf.keras.layers.Dropout(0.3)(concat)
    output = tf.keras.layers.Dense(2, activation="softmax")(dropout)
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="binary_crossentropy",
        metrics=["acc"],
    )


print(f"Strategy: {strategy}")
model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Strategy: <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7f4160bb46a0>
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)

In [ ]:
train_data = BertSemanticDataGenerator(
    train[["question1", "question2"]].values.astype("str"),
    y_train,
    batch_size=batch_size,
    shuffle=True,
)
valid_data = BertSemanticDataGenerator(
    validate[["question1", "question2"]].values.astype("str"),
    y_validate,
    batch_size=batch_size,
    shuffle=False,
)


time: 3.09 s (started: 2021-02-13 12:58:20 +00:00)


Training is done only for the top layers to perform "feature extraction", which will allow the model to use the representations of the pretrained model.

In [ ]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

Epoch 1/2
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
7580/7580 [==============================] - 3008s 394ms/step - loss: 0.4340 - acc: 0.7827 - val_loss: 0.3458 - val_acc: 0.8371
Epoch 2/2
7580/7580 [==============================] - 2986s 394ms/step - loss: 0.3479 - acc: 0.8386 - val_loss: 0.3236 - val_acc: 0.8523
time: 1h 39min 53s (started: 2021-02-13 12:59:38 +00:00)


Fine-tuning

I performed this step only after the feature extraction model has been trained to convergence on the new data.

This is an optional last step where bert_model is unfreezed and retrained with a very low learning rate. This can deliver meaningful improvement by incrementally adapting the pretrained features to the new data.

In [ ]:
# Unfreeze the bert_model.
bert_model.trainable = True
# Recompile the model to make the change effective.
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
attention_masks (InputLayer)    [(None, 128)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_bert_model_1 (TFBertModel)   ((None, 128, 768), ( 109482240   input_ids[0][0]                  
                                                                 attention_masks[0][0]      

Training the entire model end-to-end

In [ ]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

model.save('Semantic_Similarity_Fine_tuned_Bert_Quoras_Questions')

Epoch 1/2
7580/7580 [==============================] - 6910s 909ms/step - loss: 0.3077 - accuracy: 0.8621 - val_loss: 0.2637 - val_accuracy: 0.8856
Epoch 2/2
7580/7580 [==============================] - 6891s 909ms/step - loss: 0.2244 - accuracy: 0.9054 - val_loss: 0.2445 - val_accuracy: 0.8967
time: 3h 50min (started: 2021-02-13 14:39:32 +00:00)


In [ ]:
test_data = BertSemanticDataGenerator(
    test[["question1", "question2"]].values.astype("str"),
    y_test,
    batch_size=batch_size,
    shuffle=False,
)
model.evaluate(test_data, verbose=1)

2526/2526 [==============================] - 697s 276ms/step - loss: 0.2486 - accuracy: 0.8950


[0.24856306612491608, 0.8949549794197083]

time: 11min 37s (started: 2021-02-13 18:29:32 +00:00)


Inference on custom sentences

In [ ]:
labels = [1, 0]

time: 766 µs (started: 2021-02-13 18:41:10 +00:00)


In [ ]:
def check_similarity(sentence1, sentence2):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )

    proba = model.predict(test_data)[0]
    idx = np.argmax(proba)
    proba = f"{proba[idx]: .2f}%"
    pred = labels[idx]
    if pred == 1:
      label = "Similar"
    else:
      label = "Not Similar"
    print("Prediction: {0}\nProbability: {1}\n\n".format(label, proba))

time: 8.85 ms (started: 2021-02-13 18:41:10 +00:00)


In [ ]:
sentence1 = "Two women are observing something together."
sentence2 = "Two women are standing with their eyes closed."
check_similarity(sentence1, sentence2)

Prediction: Not Similar
Probability:  0.79%


time: 4.14 s (started: 2021-02-13 18:41:10 +00:00)


In [ ]:
sentence1 = "A soccer game with multiple males playing"
sentence2 = "Some men are playing a football"
check_similarity(sentence1, sentence2)

Prediction: Similar
Probability:  0.99%


time: 221 ms (started: 2021-02-13 18:41:15 +00:00)


In [ ]:
sentence1 = "How old are you?"
sentence2 = "What is your age?"
check_similarity(sentence1, sentence2)

Prediction: Similar
Probability:  0.72%


time: 247 ms (started: 2021-02-13 18:41:15 +00:00)


In [ ]:
sentence1 = "Does that dog belong to you?"
sentence2 = "Is that dog yours?"
check_similarity(sentence1, sentence2)

Prediction: Similar
Probability:  0.92%


time: 237 ms (started: 2021-02-13 18:41:15 +00:00)


In [ ]:
sentence1 = "Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?"
sentence2 = "I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?"
check_similarity(sentence1, sentence2)

Prediction: Similar
Probability:  0.62%


time: 250 ms (started: 2021-02-13 18:41:15 +00:00)


Saving and loading the model using tensorflow

In [ ]:
model.save('Semantic_Similarity_Fine_tuned_Bert_Quoras_Questions')

In [ ]:
from tensorflow import keras
model = keras.models.load_model('Semantic_Similarity_Fine_tuned_Bert_Quoras_Questions')

In [ ]:
model

Save in pickle format

In [ ]:
with open('sts_bert.pkl', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
with open('sts_bert.pkl', 'rb') as f:
    semantic_similarity_bert = pickle.load(f)